In [20]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import time
import os 
import pickle
import torch

from flare_preprocessing import *
from utilities import *

from New_SampleConstruction import *
from New_lstm2 import *
from New_logistic import *

In [2]:
# read the parquet file
high_qual_sharp = pd.read_parquet('processed_high-qual_near-center-70_no-nas_flares.parquet', engine='fastparquet')

In [ ]:
sci = pd.read_csv("D:\\2024_S1\\ML_SEP_2402\\v2_sci_20100101_20240721.csv")

In [6]:
sci.head(5)

,start_time,end_time,peak_time,label,noaa_ar,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius,log_intensity,cls,opr_cls
0,2010/5/1 1:29,2010/5/1 1:44,2010-05-01 01:39:00,C8.1,11067.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.091515,C,C
1,2010/5/1 4:58,NaN,2010-05-01 05:04:00,B1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.000000,B,NaN
2,2010/5/1 5:15,2010/5/1 5:32,2010-05-01 05:27:00,B2.1,11064.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.677781,B,B
3,2010/5/1 6:35,2010/5/1 6:47,2010-05-01 06:39:00,B1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.958607,B,NaN
4,2010/5/1 9:44,2010/5/1 10:00,2010-05-01 09:52:00,B1.2,11067.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.920819,B,B


In [7]:
high_qual_sharp

,HARPNUM,T_REC,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,...,T_FRST1,T_LAST1,NOAA_AR,NOAA_NUM,NOAA_ARS,start time,peak time,end time,flare_class,peak_intensity
0,1,2010-05-01 14:00:00+00:00,5.827622e+21,23.887000,73.347000,77.56400,31.35400,-0.151826,4.860817e+12,0.000398,...,2010-05-01 00:00:00+00:00,2010-05-11 16:12:00+00:00,11067,1,11067,NaT,NaT,NaT,None,NaN
1,1,2010-05-01 14:12:00+00:00,5.768150e+21,24.708000,71.970000,76.24900,31.19700,-0.226835,4.908301e+12,0.000081,...,2010-05-01 00:00:00+00:00,2010-05-11 16:12:00+00:00,11067,1,11067,NaT,NaT,NaT,None,NaN
2,1,2010-05-01 14:24:00+00:00,5.865157e+21,25.657000,74.254000,82.15400,32.74800,-0.260023,5.577384e+12,-0.009573,...,2010-05-01 00:00:00+00:00,2010-05-11 16:12:00+00:00,11067,1,11067,NaT,NaT,NaT,None,NaN
3,1,2010-05-01 14:36:00+00:00,5.854875e+21,25.242000,74.539000,81.89600,33.02600,-0.199640,5.290721e+12,-0.013852,...,2010-05-01 00:00:00+00:00,2010-05-11 16:12:00+00:00,11067,1,11067,NaT,NaT,NaT,None,NaN
4,1,2010-05-01 14:48:00+00:00,5.854465e+21,25.876000,75.671000,85.29700,33.55600,-0.403703,5.911234e+12,-0.012455,...,2010-05-01 00:00:00+00:00,2010-05-11 16:12:00+00:00,11067,1,11067,NaT,NaT,NaT,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3974760,11571,2024-07-20 02:24:00+00:00,1.448131e+20,27.934333,140.957333,135.03100,57.84900,0.768107,2.586889e+11,-0.016932,...,NaT,NaT,<NA>,<NA>,None,NaT,NaT,NaT,None,NaN
3974761,11571,2024-07-20 02:36:00+00:00,1.427656e+20,27.808500,140.723000,134.65225,58.10275,0.710559,2.596791e+11,-0.015961,...,NaT,NaT,<NA>,<NA>,None,NaT,NaT,NaT,None,NaN
3974762,11571,2024-07-20 02:48:00+00:00,1.407181e+20,27.682667,140.488667,134.27350,58.35650,0.653012,2.606693e+11,-0.014990,...,NaT,NaT,<NA>,<NA>,None,NaT,NaT,NaT,None,NaN
3974763,11571,2024-07-20 03:00:00+00:00,1.386705e+20,27.556833,140.254333,133.89475,58.61025,0.595465,2.616594e+11,-0.014019,...,NaT,NaT,<NA>,<NA>,None,NaT,NaT,NaT,None,NaN


In [102]:
high_qual_sharp['T_REC'].dt.tz_localize(None)

0         2010-05-01 14:00:00
1         2010-05-01 14:12:00
2         2010-05-01 14:24:00
3         2010-05-01 14:36:00
4         2010-05-01 14:48:00
                  ...        
3974760   2024-07-20 02:24:00
3974761   2024-07-20 02:36:00
3974762   2024-07-20 02:48:00
3974763   2024-07-20 03:00:00
3974764   2024-07-20 03:12:00
Name: T_REC, Length: 3974765, dtype: datetime64[ns]

In [16]:
issue_time = pd.to_datetime("2023-04-06T00:00:00Z")
valid_harps = high_qual_sharp[(high_qual_sharp['T_REC'] > issue_time - pd.Timedelta(hours=24)) & (high_qual_sharp['T_REC'] <= issue_time)]

In [17]:
harp_nums = valid_harps['HARPNUM'].unique()

In [18]:
harp_nums

<IntegerArray>
[9302, 9303, 9309, 9321, 9322, 9323, 9325, 9335, 9336, 9337, 9339, 9340, 9343,
 9344, 9345, 9346, 9347, 9350, 9351]
Length: 19, dtype: Int64

In [46]:
# load models
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model_cache = {}
def get_model(model_name, path):
    if model_name not in model_cache:
        print('new')
        model = lstm(20).to(DEVICE)
        state_dict = torch.load(path, map_location=DEVICE, weights_only=True)
        model.load_state_dict(state_dict)
        model.eval()
        model.to(DEVICE)
        model_cache[model_name] = model
    return model_cache[model_name]

In [24]:
high_qual_sharp[high_qual_sharp['HARPNUM'] == 9302][params].values

array([[2.83844200e+21, 2.54860000e+01, 8.54900000e+01, ...,
        2.34473800e+01, 2.68000000e+02, 3.64800000e+03],
       [2.86322300e+21, 2.59890000e+01, 8.42140000e+01, ...,
        2.69471610e+01, 3.08000000e+02, 3.62500000e+03],
       [2.88815100e+21, 2.55690000e+01, 8.40890000e+01, ...,
        2.68598080e+01, 3.07000000e+02, 3.62900000e+03],
       ...,
       [2.02727300e+22, 3.64090000e+01, 6.14550000e+01, ...,
        2.79403656e+02, 3.17600000e+03, 3.15040000e+04],
       [2.02707600e+22, 3.59050000e+01, 6.08860000e+01, ...,
        2.73343018e+02, 3.10700000e+03, 3.13180000e+04],
       [2.01359900e+22, 3.61970000e+01, 6.12490000e+01, ...,
        2.69741180e+02, 3.06600000e+03, 3.11740000e+04]])

In [75]:
path = "C:\\Users\\huke0\\Downloads\\0626_temp\\LSTM_models_0530"
Mplus_probs = []
uncertainties = []
uncer_lows = []
uncer_highs = []
for harpnum in harp_nums:
    print(f"Processing harp number: {harpnum}")
    input = valid_harps[valid_harps['HARPNUM'] == harpnum][params].values
    input_tensor = torch.from_numpy(input).float().to(DEVICE).unsqueeze(0)
    probs_perHARP = []
    for i in range(30):
        model_name = f"lead0_Mplus_24_whole{i}.pth"
        model_path = os.path.join(path, model_name)
        if os.path.exists(model_path):
            lstm_model = get_model(model_name, model_path)
            with torch.no_grad():
                out = lstm_model(input_tensor)
                prob = torch.sigmoid(out).cpu().numpy()
                probs_perHARP.append(prob[0][0])
        else:
            raise FileNotFoundError(f"Model file {model_path} does not exist.")
    probs_perHARP = np.array(probs_perHARP)
    mean = np.mean(probs_perHARP, axis=0)
    std = np.std(probs_perHARP, axis=0)
    Mplus_probs.append(mean)
    uncertainties.append(std)
    uncer_lows.append(mean - std)
    uncer_highs.append(mean + std)

# cut the low and high by 0 and 1
Mplus_probs = np.clip(Mplus_probs, 0, 1)
uncer_lows = np.clip(uncer_lows, 0, 1)
uncer_highs = np.clip(uncer_highs, 0, 1)

Processing harp number: 9302
Processing harp number: 9303
Processing harp number: 9309
Processing harp number: 9321
Processing harp number: 9322
Processing harp number: 9323
Processing harp number: 9325
Processing harp number: 9335
Processing harp number: 9336
Processing harp number: 9337
Processing harp number: 9339
Processing harp number: 9340
Processing harp number: 9343
Processing harp number: 9344
Processing harp number: 9345
Processing harp number: 9346
Processing harp number: 9347
Processing harp number: 9350
Processing harp number: 9351


In [106]:
Mplus_probs

array([0.6247583 , 0.63700175, 0.63791287, 0.6435562 , 0.6605689 ,
       0.6548735 , 0.655777  , 0.6411039 , 0.6675572 , 0.6618555 ,
       0.647046  , 0.65957636, 0.6769936 , 0.6485751 , 0.64738625,
       0.66893035, 0.6426137 , 0.67454875, 0.6579846 ], dtype=float32)

In [77]:
# full disk
full_disk_prob = max(Mplus_probs)

In [80]:
def full_disk2(Mplus_probs):
    full_disk_prob = 1- Mplus_probs
    full_disk_prob = np.prod(full_disk_prob)
    full_disk_prob = 1 - full_disk_prob
    return full_disk_prob

In [103]:
def genResults(issue_time, sharp_data, path = "C:\\Users\\huke0\\Downloads\\0626_temp\\LSTM_models_0530"):
    issue_time = pd.to_datetime(issue_time)
    valid_harps = sharp_data[(sharp_data['T_REC'].dt.tz_localize(None) > issue_time - pd.Timedelta(hours=24)) & (sharp_data['T_REC'].dt.tz_localize(None) <= issue_time)]
    harp_nums = valid_harps['HARPNUM'].unique()
    noaa_ars = []
    Mplus_probs = []
    uncertainties = []
    uncer_lows = []
    uncer_highs = []
    for harpnum in harp_nums:
        print(f"Processing harp number: {harpnum}")
        noaa_ar = valid_harps[valid_harps['HARPNUM'] == harpnum]['NOAA_AR'].values[0]
        noaa_ars.append(noaa_ar)
        input = valid_harps[valid_harps['HARPNUM'] == harpnum][params].values
        input_tensor = torch.from_numpy(input).float().to(DEVICE).unsqueeze(0)
        probs_perHARP = []
        for i in range(30):
            model_name = f"lead0_Mplus_24_whole{i}.pth"
            model_path = os.path.join(path, model_name)
            if os.path.exists(model_path):
                lstm_model = get_model(model_name, model_path)
                with torch.no_grad():
                    out = lstm_model(input_tensor)
                    prob = torch.sigmoid(out).cpu().numpy()
                    probs_perHARP.append(prob[0][0])
            else:
                raise FileNotFoundError(f"Model file {model_path} does not exist.")
        probs_perHARP = np.array(probs_perHARP)
        mean = np.mean(probs_perHARP, axis=0)
        std = np.std(probs_perHARP, axis=0)
        Mplus_probs.append(mean)
        uncertainties.append(std)
        uncer_lows.append(mean - std)
        uncer_highs.append(mean + std)

    # cut the low and high by 0 and 1
    Mplus_probs = np.clip(Mplus_probs, 0, 1)
    uncer_lows = np.clip(uncer_lows, 0, 1)
    uncer_highs = np.clip(uncer_highs, 0, 1)
    full_disk_prob = max(Mplus_probs)
    return harp_nums,noaa_ars, Mplus_probs, uncertainties, uncer_lows, uncer_highs, full_disk_prob

In [107]:
import json
from datetime import timedelta

def genJson(issue_time, sharp_data, path = "C:\\Users\\huke0\\Downloads\\0626_temp\\LSTM_models_0530"):

    issue_time = pd.to_datetime(issue_time)
    start_time = issue_time.replace(minute=0, second=0)
    end_time = start_time + pd.Timedelta(hours=24)

    harp_nums,noaa_ars, Mplus_probs, uncertainties, uncer_lows, uncer_highs, full_disk_prob = genResults(issue_time, sharp_data, path)

    region_forecasts = []
    for harp, ar, prob, std, low, high in zip(harp_nums,  noaa_ars, Mplus_probs, uncertainties, uncer_lows, uncer_highs):
        region_entry = {
            "prediction_window":{
                "start_time": start_time.strftime("%Y-%m-%dT%H:%MZ"),
                "end_time": end_time.strftime("%Y-%m-%dT%H:%MZ")
            },
            "region_ids":[
                {"type": "HARP", "number": str(harp)},
                {"type": "NOAA", "number": str(ar)}
            ],
            "flare_probabilities":[
                {
                    "class": "m+",
                    "probability": float(prob),
                    "uncertainty": float(std),
                    "uncertainty_low": float(low),
                    "uncertainty_high": float(high)
                }
            ]
        }
        region_forecasts.append(region_entry)

    full_disk_forecast = {
        "prediction_window":{
                "start_time": start_time.strftime("%Y-%m-%dT%H:%MZ"),
                "end_time": end_time.strftime("%Y-%m-%dT%H:%MZ")
            },
        "flare_probabilities":[
            {
                "class": "m+",
                "probability": float(full_disk_prob)
            }
        ]
    }

    # JSON structure
    forecast_submission = {
        "forecast_submission": {
            "model": {
                "short_name": "CLEARflare-LSTM",  
                "spase_id": "----"
            },
            "issue_time": issue_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "mode": "forecast",
            "full_disk_forecasts": [full_disk_forecast],
            "region_forecasts": region_forecasts
        }
    }

    # save to JSON file
    with open(f"CLEARflare-LSTM{issue_time.strftime('%Y%m%dT%H%MZ')}.json", 'w') as f:
        json.dump(forecast_submission, f, indent=2)

In [108]:
genJson("2023-04-06", high_qual_sharp, path="C:\\Users\\huke0\\Downloads\\0626_temp\\LSTM_models_0530")

Processing harp number: 9302
Processing harp number: 9303
Processing harp number: 9309
Processing harp number: 9321
Processing harp number: 9322
Processing harp number: 9323
Processing harp number: 9325
Processing harp number: 9335
Processing harp number: 9336
Processing harp number: 9337
Processing harp number: 9339
Processing harp number: 9340
Processing harp number: 9343
Processing harp number: 9344
Processing harp number: 9345
Processing harp number: 9346
Processing harp number: 9347
Processing harp number: 9350
Processing harp number: 9351


In [81]:
full_disk2(Mplus_probs)

0.9999999981884634

In [83]:
sci['peak_time'] = pd.to_datetime(sci['peak_time'])

In [87]:
sci['peak_time'].dt.tz_localize(None)

0       2010-05-01 01:39:00
1       2010-05-01 05:04:00
2       2010-05-01 05:27:00
3       2010-05-01 06:39:00
4       2010-05-01 09:52:00
                ...        
30733   2024-07-21 16:31:00
30734   2024-07-21 20:12:00
30735   2024-07-21 20:27:00
30736   2024-07-21 21:56:00
30737   2024-07-21 23:28:00
Name: peak_time, Length: 30738, dtype: datetime64[ns]

In [90]:
issue_time.tz_localize(None)

Timestamp('2023-04-06 00:00:00')

In [91]:
sci[(sci['peak_time'] > issue_time.tz_localize(None) - pd.Timedelta(hours=24)) & (sci['peak_time'] <= issue_time.tz_localize(None))]

,start_time,end_time,peak_time,label,noaa_ar,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius,log_intensity,cls,opr_cls


In [94]:
issue_time.tz_localize(None) - pd.Timedelta(hours=24)

Timestamp('2023-04-05 00:00:00')

In [95]:
issue_time.tz_localize(None)

Timestamp('2023-04-06 00:00:00')

In [92]:
sci[(sci['peak_time'] > issue_time.tz_localize(None) - pd.Timedelta(hours=24))]

,start_time,end_time,peak_time,label,noaa_ar,hg1,hg2,car1,car2,rtheta1,rtheta2,xy1,xy2,solar_p_angle,solar_radius,log_intensity,cls,opr_cls
26434,2023/4/7 2:35,2023/4/7 3:00,2023-04-07 02:49:00,C1.3,13270.0,55.221603,-13.370833,233.952220,-13.370833,0.819137,258.001620,768.02000,-163.22472,-26.262005,958.53810,-5.886057,C,C
26435,2023/4/7 6:21,2023/4/7 6:47,2023-04-07 06:27:00,C1.4,13272.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,1.002933,106.327900,-922.53760,-270.25552,-26.262056,958.49630,-5.853872,C,C
26436,2023/4/7 16:27,2023/4/7 16:41,2023-04-07 16:37:00,C1.1,13272.0,282.786650,-9.203439,93.928850,-9.203439,0.973187,98.012610,-923.57720,-130.00717,-26.261076,958.37930,-5.958607,C,C
26437,2023/4/7 20:14,2023/4/7 20:27,2023-04-07 20:22:00,C1.7,13272.0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,1.020396,108.897354,-925.17554,-316.70935,-26.260262,958.33606,-5.769551,C,C
26438,2023/4/7 20:33,2023/4/7 20:46,2023-04-07 20:42:00,C2.5,13272.0,292.984600,-16.286722,101.881370,-16.286722,0.917012,105.108350,-848.42550,-229.05478,-26.260181,958.33220,-5.602060,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30733,2024/7/21 16:18,2024/7/21 16:59,2024-07-21 16:31:00,M1.0,13757.0,337.472960,18.971746,49.162434,18.971746,0.440999,55.583748,-343.47568,235.32564,6.722910,944.12665,-5.000000,M,M
30734,2024/7/21 20:04,NaN,2024-07-21 20:12:00,C4.7,13744.0,68.252680,17.042410,137.911620,17.042410,0.927257,286.390800,839.87940,247.04288,6.788609,944.13780,-5.327902,C,C
30735,2024/7/21 20:21,2024/7/21 20:33,2024-07-21 20:27:00,C4.5,13744.0,61.154408,5.928951,130.675540,5.928951,0.875330,274.019800,824.39954,57.93365,6.793064,944.13860,-5.346787,C,C
30736,2024/7/21 21:47,2024/7/21 22:04,2024-07-21 21:56:00,C5.4,13762.0,21.028952,-8.493025,89.732360,-8.493025,0.423262,237.351010,336.47590,-215.59070,6.819478,944.14310,-5.267606,C,C
